# Use case

In the basics use case we demonstrated how to simulate a ship's performance in a single condition at a single speed.

We will extend this example and simulate a ship's performance over multiple speeds. This will allow us to construct a speed-fuel table and speed-fuel curve, showing the required fuel consumption for each speed of the ship.

## Authorize

Let's first make sure we're authorized to use the API. Fill in your API key and run the following code.

If everything is alright, you should see a list of ships.


In [20]:
API_KEY = "your-api-key"

In [22]:
import json
import requests

API_URL = "https://api.toqua.ai"

url = "https://api.toqua.ai/ships/"
headers = {"accept": "application/json", "X-API-Key": API_KEY}
response = requests.get(url, headers=headers)

print(json.dumps(response.json(), indent=2))


[
  {
    "name": "Trial Vessel",
    "imo_number": 9999999,
    "type": "Tanker",
    "class": null,
    "country": "SC",
    "build_year": 2015,
    "shipyard": "Toqua Shipyard",
    "dwt": 220000.0,
    "beam": 55.0,
    "loa": 300.0,
    "mcr": null,
    "max_rpm": null,
    "uuid": "eycrYbrzJNsJecGqKraUCn"
  }
]


Fill in the IMO number of the ship you want to analyze.


In [23]:
IMO_NUMBER = 9999999

## Conditioning parameters

Let's again define our conditions.


In [53]:
wind_speed = 6          # [m/s]
wind_direction = 180    # [degrees]
wave_height = 2         # [m]
wave_direction = 90     # [degrees]
current_speed = 2       # [m/s]
current_direction = 0   # [degrees]
mean_draft = 20         # [m]
trim = -1               # [m]
ship_heading = 0        # [degrees]
fuel_specific_energy  = 41.5 # [MJ/kg]

Our entrypoint will this time be a list, rather than a single value. We will analyze the ship's fuel consumption in speeds ranging from a STW of 6 knots to 16 knots.

In [54]:
stw = list(range(6, 17))

print(stw)

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


## Define the API input


Remember that the Toqua API expects the model input to look like this:

```
{
     "date": "...",
     "data": {
          "stw": [...],
          "draft_avg": [...],
          "trim": [...],
          "wave_direction": [...],
          "wave_height": [...],
          "wave_period": [...],
          "current_speed": [...],
          "current_direction": [...],
          "wind_direction": [...],
          "wind_speed": [...],
          "ship_heading": [...],
          "fuel_specific_energy": [...]
     }
}
```

We will again ignore the `date` parameter for now.

Each parameter expects a list of values and all lists must have exactly the same length.
The parameter value at each index $i$ corresponds to $$Condition_i = \{stw_i, wave\_direction_i, wave\_speed_i, draft\_avg_i, ...\}$$

Only the `stw` parameter is currently a list and we are only simulating a single condition, so we will have to duplicate the conditioning parameters once for each element in the `stw` list.

In [61]:
length_input = len(stw)

api_input = {
    "data": {
        "stw": stw,
        "wave_direction": [wave_direction]*length_input,
        "wave_height": [wave_height]*length_input,
        "wind_direction": [wind_direction]*length_input,
        "wind_speed": [wind_speed]*length_input,
        "current_direction": [current_direction]*length_input,
        "current_speed": [current_speed]*length_input,
        "draft_avg": [mean_draft]*length_input,
        "trim": [trim]*length_input,
        "fuel_specific_energy": [fuel_specific_energy]*length_input
    }
}

## Query the API


In [62]:
def query_api(imo_number, payload):
    url = f"https://api.toqua.ai/ships/{imo_number}/models/latest/predict"
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "X-API-Key": API_KEY,
    }
    response = requests.post(url, json=payload, headers=headers)
    return response


Let's look at the values the model predicts.

In [63]:
response = query_api(IMO_NUMBER, api_input)

print(response.json())


{'sog': [2.11232, 3.11232, 4.11232, 5.11232, 6.11232, 7.11232, 8.11232, 9.11232, 10.11232, 11.11232, 12.11232], 'stw': [6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0], 'me_rpm': [22.54890544589254, 24.97097532128466, 27.46916405829512, 30.06564265987471, 32.777084317488104, 35.61669598712825, 38.595466765573974, 41.722914386818964, 45.007600274878456, 48.45748412225273, 52.08011540213334], 'me_power': [1869.7667968750002, 2346.1419921875, 2944.439453125, 3692.91328125, 4625.341796875, 5781.926562500001, 7210.34765625, 8966.95703125, 11118.15859375, 13741.9921875, 16929.89375], 'me_fo_consumption': [8.798402460492445, 10.897780077856206, 13.468419573257291, 16.596091186292796, 20.38510203019365, 24.97668807167932, 30.590286700179377, 37.60967357895273, 46.75442557658038, 59.40863917231829, 78.2326173179663], 'me_fo_emission': [27.895335000991295, 34.5514117368431, 42.701624257012234, 52.6179071061413, 64.63096598672897, 79.18858953125927, 96.9865039829187, 119.24147008206

## Speed-Fuel Table

Using the Pandas library we can transform this output into table format to make it easier on the eyes, and for future data transformations.

In [64]:
import pandas as pd

df = pd.DataFrame(response.json())
df

,sog,stw,me_rpm,me_power,me_fo_consumption,me_fo_emission
0,2.11232,6.0,22.548905,1869.766797,8.798402,27.895335
1,3.11232,7.0,24.970975,2346.141992,10.897780,34.551412
2,4.11232,8.0,27.469164,2944.439453,13.468420,42.701624
3,5.11232,9.0,30.065643,3692.913281,16.596091,52.617907
4,6.11232,10.0,32.777084,4625.341797,20.385102,64.630966
5,7.11232,11.0,35.616696,5781.926563,24.976688,79.188590
6,8.11232,12.0,38.595467,7210.347656,30.590287,96.986504
7,9.11232,13.0,41.722914,8966.957031,37.609674,119.241470
8,10.11232,14.0,45.007600,11118.158594,46.754426,148.234906
9,11.11232,15.0,48.457484,13741.992188,59.408639,188.355090


There we have our speed-fuel table. For speeds ranging from 6 to 16 knots it tells us the predicted fuel consumption in the conditions we defined earlier.

# Speed-Fuel Curve

Finally, to make things more tangible we can use the Plotly library to visualize our table as a speed-fuel curve.

In [65]:
import plotly.express as px

fig = px.line(df, x="stw", y="me_fo_consumption", title='Speed-Fuel Curve')
fig.update_layout(xaxis_title='Speed Through Water [kn]',
                   yaxis_title='Fuel Consumption [mt/day]')
fig.show()